In [33]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config('spark.shuffle.useOldFetchProtocol', 'true'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [34]:
df=spark.read \
.format("csv") \
.option("inferSchema",True) \
.option("header","true") \
.load("/user/itv011656/project/US_births_2000-2014_SSA.csv")

In [3]:
display(df)

year,month,date_of_month,day_of_week,births
2000,1,1,6,9083
2000,1,2,7,8006
2000,1,3,1,11363
2000,1,4,2,13032
2000,1,5,3,12558
2000,1,6,4,12466
2000,1,7,5,12516
2000,1,8,6,8934
2000,1,9,7,7949
2000,1,10,1,11668


In [8]:
df.withColumn("sum of births", sum("births").over(window_spec)).show()

+----+-----+-------------+-----------+------+-------------+
|year|month|date_of_month|day_of_week|births|sum of births|
+----+-----+-------------+-----------+------+-------------+
|2003|    1|            1|          3|  7908|      4163060|
|2003|    1|            2|          4| 11124|      4163060|
|2003|    1|            3|          5| 12252|      4163060|
|2003|    1|            4|          6|  8519|      4163060|
|2003|    1|            5|          7|  7495|      4163060|
|2003|    1|            6|          1| 11364|      4163060|
|2003|    1|            7|          2| 12657|      4163060|
|2003|    1|            8|          3| 12167|      4163060|
|2003|    1|            9|          4| 12081|      4163060|
|2003|    1|           10|          5| 12517|      4163060|
|2003|    1|           11|          6|  8471|      4163060|
|2003|    1|           12|          7|  7297|      4163060|
|2003|    1|           13|          1| 11197|      4163060|
|2003|    1|           14|          2| 1

# Total birth on day 6 of every week

In [35]:
new_df=df.filter("day_of_week=6").groupBy("year","day_of_week").agg(sum("births")).orderBy(desc("year"))

In [36]:
new_df.show()

+----+-----------+-----------+
|year|day_of_week|sum(births)|
+----+-----------+-----------+
|2014|          6|     434881|
|2013|          6|     426992|
|2012|          6|     426211|
|2011|          6|     428497|
|2010|          6|     420764|
|2009|          6|     437040|
|2008|          6|     453269|
|2007|          6|     464142|
|2006|          6|     460500|
|2005|          6|     454600|
|2004|          6|     448659|
|2003|          6|     453943|
|2002|          6|     452840|
|2001|          6|     462516|
|2000|          6|     479641|
+----+-----------+-----------+



In [7]:
window_spec=Window.partitionBy("year").orderBy("year")

In [37]:
from pyspark.sql.functions import *
from pyspark.sql import *

In [32]:
spark.stop()

# Checking null values for births, day_of_week,month

In [11]:
null_check_result = df.filter(df.day_of_week.isNull()).count()

In [ ]:
null_check_result = df.filter(df.births.isNull()).count()

In [45]:
null_check_result = df.filter(df.month.isNull()).count()

In [46]:
display(null_check_result)

0

# max and min births on each day. 

In [13]:
df.groupBy("year","day_of_week").agg(max("births"),min("births")).orderBy("year").show()

+----+-----------+-----------+-----------+
|year|day_of_week|max(births)|min(births)|
+----+-----------+-----------+-----------+
|2000|          2|      13991|       9550|
|2000|          5|      13658|       9805|
|2000|          7|       8683|       6971|
|2000|          4|      13900|       8144|
|2000|          1|      12974|       6719|
|2000|          3|      13917|      12077|
|2000|          6|       9860|       8367|
|2001|          2|      14063|       6603|
|2001|          5|      14235|       9594|
|2001|          4|      14151|       7513|
|2001|          3|      14055|       9608|
|2001|          6|       9662|       8100|
|2001|          1|      12709|       7626|
|2001|          7|       8495|       7212|
|2002|          7|       8433|       7083|
|2002|          3|      13969|       6774|
|2002|          6|       9613|       8159|
|2002|          5|      14015|       9341|
|2002|          4|      14247|       7392|
|2002|          2|      14241|       7927|
+----+-----

In [14]:
spark.sql("create database if not exists usbirthdata")

""


In [39]:
spark.sql("USE usbirthdata")

""


In [41]:
df.createOrReplaceTempView("us_birth_details")

In [42]:
spark.sql("show tables")

database,tableName,isTemporary
,us_birth_details,true


# yoy birth

In [59]:
yoy_birth=spark.sql("""Select year, total_birth,lead(total_birth,1) over(order By year) as yoy_birth
from (Select year,sum(births) as total_birth from us_birth_details group by year order by year)""")

# difference between total birth and yoy birth

In [60]:
yoy_birth.withColumn("difference",col("total_birth")-col("yoy_birth")).show() 

+----+-----------+---------+----------+
|year|total_birth|yoy_birth|difference|
+----+-----------+---------+----------+
|2000|    4149598|  4110963|     38635|
|2001|    4110963|  4099313|     11650|
|2002|    4099313|  4163060|    -63747|
|2003|    4163060|  4186863|    -23803|
|2004|    4186863|  4211941|    -25078|
|2005|    4211941|  4335154|   -123213|
|2006|    4335154|  4380784|    -45630|
|2007|    4380784|  4310737|     70047|
|2008|    4310737|  4190991|    119746|
|2009|    4190991|  4055975|    135016|
|2010|    4055975|  4006908|     49067|
|2011|    4006908|  4000868|      6040|
|2012|    4000868|  3973337|     27531|
|2013|    3973337|  4010532|    -37195|
|2014|    4010532|     null|      null|
+----+-----------+---------+----------+



In [58]:
spark.sql("Select year,sum(births) as total_birth from us_birth_details group by year order by year")

year,total_birth
2000,4149598
2001,4110963
2002,4099313
2003,4163060
2004,4186863
2005,4211941
2006,4335154
2007,4380784
2008,4310737
2009,4190991


# mom birth rate

In [43]:
spark.sql("""Select month,total_birth,lead(total_birth,1) over(order By month) as mom_birth
from (Select month,sum(births) as total_birth from us_birth_details group by year,month)""").show()

+-----+-----------+---------+
|month|total_birth|mom_birth|
+-----+-----------+---------+
|    1|     342379|   360521|
|    1|     360521|   361868|
|    1|     361868|   342975|
|    1|     342975|   336163|
|    1|     336163|   329962|
|    1|     329962|   337478|
|    1|     337478|   320661|
|    1|     320661|   337739|
|    1|     337739|   345976|
|    1|     345976|   337472|
|    1|     337472|   339077|
|    1|     339077|   327352|
|    1|     327352|   324845|
|    1|     324845|   328120|
|    1|     328120|   312789|
|    2|     312789|   294882|
|    2|     294882|   301927|
|    2|     301927|   309819|
|    2|     309819|   321348|
|    2|     321348|   300899|
+-----+-----------+---------+
only showing top 20 rows

